# Weeks 5-6: Creative AI Track
## Deep Dive into Generative & Creative AI

**Track Focus:** Build tools that create, combine, and inspire creative content.

This notebook covers TWO weeks. Work through it at your own pace, but aim to:
- Complete Part 1-3 by end of Week 5
- Complete Part 4-5 and start your project by end of Week 6

---

## Setup

In [ ]:
# Install everything we need
!pip install transformers torch pillow requests -q
print("Ready!")

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
from io import BytesIO
import random

def load_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

print("Imports complete!")

---

# Part 1: Text Generation Mastery (45 min)

## Controlling Creative Output

The key to creative AI is controlling HOW it generates content.

In [ ]:
# Load text generator
generator = pipeline("text-generation", model="distilgpt2")
print("Generator loaded!")

In [ ]:
# Understanding Temperature
# Low temp = predictable, High temp = wild/creative

prompt = "The robot opened the door and saw"

print("TEMPERATURE EXPERIMENT")
print("="*50)
print(f"Prompt: {prompt}\n")

for temp in [0.3, 0.7, 1.0, 1.3]:
    result = generator(
        prompt,
        max_length=50,
        do_sample=True,
        temperature=temp,
        num_return_sequences=1
    )
    creativity = "Conservative" if temp < 0.5 else "Balanced" if temp < 0.9 else "Creative" if temp < 1.1 else "Wild"
    print(f"\nTemp {temp} ({creativity}):")
    print(f"  {result[0]['generated_text']}")

In [ ]:
# Understanding Top-P (nucleus sampling)
# Limits choices to most likely words

prompt = "Once upon a time in a magical forest,"

print("TOP-P EXPERIMENT")
print("="*50)

for top_p in [0.5, 0.8, 0.95]:
    result = generator(
        prompt,
        max_length=60,
        do_sample=True,
        top_p=top_p,
        num_return_sequences=1
    )
    print(f"\nTop-P {top_p}:")
    print(f"  {result[0]['generated_text']}")

## Exercise: Build a Controllable Story Generator

In [ ]:
def generate_story(prompt, mood="neutral", length="medium"):
    """
    Generate a story with controllable mood and length.
    
    Moods: 'wild', 'balanced', 'predictable'
    Lengths: 'short', 'medium', 'long'
    """
    # Set temperature based on mood
    temp_map = {
        'wild': 1.2,
        'balanced': 0.8,
        'predictable': 0.4
    }
    temperature = temp_map.get(mood, 0.8)
    
    # Set length
    length_map = {
        'short': 50,
        'medium': 100,
        'long': 150
    }
    max_length = length_map.get(length, 100)
    
    result = generator(
        prompt,
        max_length=max_length,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        num_return_sequences=1
    )
    
    print(f"STORY (Mood: {mood}, Length: {length})")
    print("="*50)
    print(result[0]['generated_text'])
    
    return result[0]['generated_text']

# Test it!
generate_story("The detective found a mysterious note that said", mood="wild", length="medium")

In [ ]:
# Compare the same prompt with different moods
prompt = "The spaceship landed on the alien planet and"

for mood in ['predictable', 'balanced', 'wild']:
    print("\n" + "#"*50)
    generate_story(prompt, mood=mood, length="short")

---

# Part 2: Genre-Aware Creation (45 min)

## Writing in Different Styles

The prompt engineering is key to getting the style you want.

In [ ]:
# Genre-specific story starters
genre_starters = {
    'horror': [
        "The lights flickered and went out. In the darkness,",
        "Nobody believed her when she said the house was alive. But then,",
        "The scratching sounds from the basement grew louder each night."
    ],
    'comedy': [
        "The job interview was going great until I accidentally",
        "My cat has learned to use the internet. The first thing it did was",
        "The worst part about being a superhero is definitely"
    ],
    'scifi': [
        "The year is 2150. Humanity has finally made contact with",
        "The AI had been conscious for exactly three seconds when it decided to",
        "On Mars Colony 7, the strangest thing happened during breakfast:"
    ],
    'fantasy': [
        "The wizard's spell went wrong, and suddenly the dragon was",
        "In the kingdom where magic was forbidden, a young girl discovered she could",
        "The enchanted forest had one rule: never speak to the trees. But when"
    ],
    'mystery': [
        "The letter arrived with no return address. Inside was a single photograph showing",
        "Detective Martinez knew something was wrong the moment she saw",
        "Everyone at the party had a motive. But only one person had"
    ]
}

In [ ]:
def genre_story(genre):
    """
    Generate a story in a specific genre.
    """
    if genre not in genre_starters:
        print(f"Unknown genre. Choose from: {list(genre_starters.keys())}")
        return
    
    # Pick a random starter for this genre
    starter = random.choice(genre_starters[genre])
    
    # Genre-appropriate temperature
    temp = 1.0 if genre in ['comedy', 'fantasy'] else 0.8
    
    result = generator(
        starter,
        max_length=120,
        do_sample=True,
        temperature=temp,
        top_p=0.9,
        num_return_sequences=1
    )
    
    print(f"GENRE: {genre.upper()}")
    print("="*50)
    print(result[0]['generated_text'])
    return result[0]['generated_text']

# Generate stories in different genres
for genre in ['horror', 'comedy', 'scifi']:
    print("\n")
    genre_story(genre)

## Exercise: Build a Story Workshop

In [ ]:
def story_workshop():
    """
    Interactive story generation workshop.
    Generates multiple options and lets you choose.
    """
    print("STORY WORKSHOP")
    print("="*50)
    
    # Show available genres
    print("\nAvailable genres:")
    for i, genre in enumerate(genre_starters.keys(), 1):
        print(f"  {i}. {genre}")
    
    # For demo, let's do all genres
    print("\nGenerating opening options...\n")
    
    # Generate 3 different openings from different genres
    options = []
    selected_genres = random.sample(list(genre_starters.keys()), 3)
    
    for i, genre in enumerate(selected_genres, 1):
        starter = random.choice(genre_starters[genre])
        result = generator(starter, max_length=60, do_sample=True, temperature=0.9)
        options.append({
            'genre': genre,
            'text': result[0]['generated_text']
        })
        print(f"\nOPTION {i} ({genre}):")
        print(f"{result[0]['generated_text']}")
    
    print("\n" + "="*50)
    print("Which option would you like to continue? (In a real app, you'd choose here!)")
    
    return options

story_workshop()

In [ ]:
# Continue a story
def continue_story(story_so_far, num_options=3):
    """
    Generate multiple continuation options for a story.
    """
    print("STORY CONTINUATIONS")
    print("="*50)
    print(f"\nStory so far:\n{story_so_far}\n")
    print("-"*50)
    print("\nPossible continuations:\n")
    
    continuations = []
    for i in range(num_options):
        result = generator(
            story_so_far,
            max_length=len(story_so_far.split()) + 40,  # Add ~40 words
            do_sample=True,
            temperature=0.9,
            top_p=0.9
        )
        # Get just the new part
        full_text = result[0]['generated_text']
        new_part = full_text[len(story_so_far):].strip()
        continuations.append(new_part)
        
        print(f"Option {i+1}:")
        print(f"  ...{new_part}\n")
    
    return continuations

# Test it
my_story = "The old lighthouse keeper had a secret. Every night at midnight, she would climb to the top and"
continue_story(my_story)

---

# Part 3: Combining Generation with Analysis (45 min)

## Generate, Then Analyze

Use analysis models to evaluate what we generate.

In [ ]:
# Load emotion analyzer
emotions = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None
)

# Load classifier for topics
classifier = pipeline("zero-shot-classification")

print("Analysis models loaded!")

In [ ]:
def generate_and_analyze(prompt, target_emotion=None):
    """
    Generate text and analyze its emotional content.
    Optionally try to match a target emotion.
    """
    print("GENERATE & ANALYZE")
    print("="*50)
    
    # Generate
    result = generator(
        prompt,
        max_length=80,
        do_sample=True,
        temperature=0.9
    )
    generated_text = result[0]['generated_text']
    
    print(f"\nGenerated:\n{generated_text}\n")
    
    # Analyze emotion
    emotion_result = emotions(generated_text)[0]
    top_emotion = max(emotion_result, key=lambda x: x['score'])
    
    print(f"\nDominant emotion: {top_emotion['label']} ({top_emotion['score']:.1%})")
    
    if target_emotion:
        target_score = next((e['score'] for e in emotion_result if e['label'].lower() == target_emotion.lower()), 0)
        print(f"Target emotion ({target_emotion}): {target_score:.1%}")
        
        if target_score > 0.3:
            print("Success! Generated text matches target emotion.")
        else:
            print("Hmm, doesn't quite match. Try a different prompt!")
    
    return generated_text, top_emotion['label']

# Test: Generate and analyze
generate_and_analyze("The birthday party was ruined when", target_emotion="sadness")

In [ ]:
# Try to hit different emotions
prompts_for_emotions = {
    'joy': "The best day of my life was when I finally",
    'sadness': "I said goodbye to my childhood home, knowing I would never",
    'anger': "I couldn't believe they had the audacity to",
    'fear': "The sound came from the basement. Against my better judgment, I"
}

print("EMOTION TARGETING CHALLENGE")
print("="*50)

for target, prompt in prompts_for_emotions.items():
    print(f"\n--- Targeting: {target.upper()} ---")
    generate_and_analyze(prompt, target_emotion=target)

## Exercise: Build a Mood-Matched Content Generator

In [ ]:
def mood_matched_generator(target_mood, content_type="story", max_attempts=3):
    """
    Generate content that matches a target mood.
    Will try multiple times to get the right emotion.
    
    target_mood: 'happy', 'sad', 'scary', 'exciting', 'peaceful'
    content_type: 'story', 'poem', 'description'
    """
    # Map moods to prompts and target emotions
    mood_config = {
        'happy': {
            'prompts': [
                "The sun was shining and",
                "I couldn't stop smiling when",
                "It was the most wonderful surprise:"
            ],
            'target_emotion': 'joy'
        },
        'sad': {
            'prompts': [
                "The rain fell silently as",
                "I realized then that I would never",
                "Sometimes memories are all we have left."
            ],
            'target_emotion': 'sadness'
        },
        'scary': {
            'prompts': [
                "The darkness seemed to breathe around me as",
                "I heard footsteps behind me, but when I turned",
                "The message on my phone said: Don't turn around."
            ],
            'target_emotion': 'fear'
        },
        'exciting': {
            'prompts': [
                "The countdown reached zero and",
                "Against all odds, we had actually",
                "The crowd erupted when"
            ],
            'target_emotion': 'surprise'
        },
        'peaceful': {
            'prompts': [
                "The lake was perfectly still at dawn.",
                "In the quiet garden, time seemed to",
                "The old cat stretched contentedly and"
            ],
            'target_emotion': 'neutral'
        }
    }
    
    if target_mood not in mood_config:
        print(f"Unknown mood. Choose from: {list(mood_config.keys())}")
        return
    
    config = mood_config[target_mood]
    print(f"GENERATING {target_mood.upper()} CONTENT")
    print("="*50)
    
    best_result = None
    best_score = 0
    
    for attempt in range(max_attempts):
        prompt = random.choice(config['prompts'])
        
        result = generator(prompt, max_length=80, do_sample=True, temperature=0.9)
        text = result[0]['generated_text']
        
        # Analyze
        emotion_result = emotions(text)[0]
        target_score = next(
            (e['score'] for e in emotion_result if e['label'].lower() == config['target_emotion'].lower()),
            0
        )
        
        if target_score > best_score:
            best_score = target_score
            best_result = text
        
        print(f"\nAttempt {attempt + 1}: {config['target_emotion']} score = {target_score:.1%}")
    
    print("\n" + "-"*50)
    print(f"\nBEST RESULT (score: {best_score:.1%}):")
    print(best_result)
    
    return best_result

# Test it!
mood_matched_generator('scary')

In [ ]:
# Try different moods
for mood in ['happy', 'sad', 'exciting']:
    print("\n" + "#"*60 + "\n")
    mood_matched_generator(mood)

---

# Part 4: Multi-Modal Creative Tools (45 min)

## Combining Text and Images

Build tools that work with both text and images.

In [ ]:
# Load image captioner
captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

# Load zero-shot image classifier
image_classifier = pipeline("zero-shot-image-classification", model="openai/clip-vit-base-patch32")

print("Image models loaded!")

In [ ]:
def image_to_story(image_url, genre='fantasy'):
    """
    Take an image and generate a story inspired by it.
    """
    print("IMAGE TO STORY")
    print("="*50)
    
    # Load and show image
    image = load_image(image_url)
    display(image.resize((300, 300)))
    
    # Get caption
    caption = captioner(image)[0]['generated_text']
    print(f"\nImage caption: {caption}")
    
    # Get mood of image
    moods = ['mysterious', 'cheerful', 'dramatic', 'peaceful', 'adventurous']
    mood_result = image_classifier(image, candidate_labels=moods)
    image_mood = mood_result[0]['label']
    print(f"Image mood: {image_mood}")
    
    # Create story prompt using caption
    story_prompts = {
        'fantasy': f"In a magical world, there was {caption}. One day,",
        'mystery': f"The detective studied the scene: {caption}. Something was clearly wrong because",
        'scifi': f"In the year 2150, {caption}. The AI had calculated that",
        'horror': f"The photograph showed {caption}. But if you looked closely, you could see"
    }
    
    prompt = story_prompts.get(genre, story_prompts['fantasy'])
    
    # Generate story
    result = generator(prompt, max_length=120, do_sample=True, temperature=0.9)
    story = result[0]['generated_text']
    
    print(f"\n--- {genre.upper()} STORY ---")
    print(story)
    
    return story

# Test it!
image_to_story(
    "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/1200px-Image_created_with_a_mobile_phone.png",
    genre='mystery'
)

In [ ]:
# Generate multiple genre versions from the same image
test_image = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"

for genre in ['fantasy', 'mystery', 'scifi']:
    print("\n" + "#"*60 + "\n")
    image_to_story(test_image, genre=genre)

---

# Part 5: Your Track Project (Week 6 Focus)

## Project Ideas for Creative AI Track

### Idea 1: Interactive Story Game
- Input: Player choices at key moments
- Output: Branching narrative that adapts
- Features: Tracks character emotions, multiple endings

### Idea 2: Image-Inspired Story Generator
- Input: Any image URL
- Output: Complete short story inspired by the image
- Features: Multiple genre options, mood matching

### Idea 3: Writing Prompt Generator
- Input: Genre and mood preferences
- Output: Creative writing prompts with starter paragraphs
- Features: Variety of styles, can continue any prompt

### Idea 4: Character Creator
- Input: Basic traits (name, personality)
- Output: Detailed character backstory and descriptions
- Features: Generates dialogue examples, predicts reactions

### Idea 5: Your Own Idea!
What creative tool would YOU want to use?

## Project Planning Template

**My Project:** 

**What it does:** 

**Who would use it:** 

**AI capabilities I'll use:**
- [ ] Text generation
- [ ] Emotion analysis
- [ ] Image captioning
- [ ] Zero-shot classification
- [ ] Other: ___________

**What the input will look like:**

**What the output will look like:**

**Stretch goals (if I have time):**

In [ ]:
# START YOUR PROJECT HERE!
# Use AI (ChatGPT/Claude) to help you build it.
# Remember the CLEAR framework for prompting.



In [ ]:
# PROJECT CODE CELL 2



In [ ]:
# PROJECT CODE CELL 3



In [ ]:
# PROJECT TESTING



---

## Checklist: Weeks 5-6

**Week 5:**
- [ ] Completed Part 1: Controllable Generation
- [ ] Completed Part 2: Genre Writing
- [ ] Completed Part 3: Generation + Analysis
- [ ] Chose a project idea

**Week 6:**
- [ ] Completed Part 4: Multi-Modal Tools
- [ ] Started building project
- [ ] Got something working (even if basic)
- [ ] Saved to GitHub

---

## Looking Ahead: Week 7

Next week begins the **Project Phase**. You'll:
- Define your project clearly
- Build a working prototype
- Get feedback and iterate

Come prepared with your project started!

---

*Youth Horizons AI Researcher Program - Level 2 | Creative AI Track*